### Load dataset

In [1]:
import pandas as pd

RANDOM_STATE = 42

FILE_PATH = "../Data/patient_genes_featureSet_2.csv" # Can be replaced with desired variant for different feature sets
variant = 'svm'
df = pd.read_csv(FILE_PATH)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977 entries, 0 to 976
Data columns (total 32 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   FYN      977 non-null    int64 
 1   BRCA1    977 non-null    int64 
 2   FOXC1    977 non-null    int64 
 3   TBC1D1   977 non-null    int64 
 4   LAG3     977 non-null    int64 
 5   CDK6     977 non-null    int64 
 6   GATA3    977 non-null    int64 
 7   CCND1    977 non-null    int64 
 8   PRR4     977 non-null    int64 
 9   EPCAM    977 non-null    int64 
 10  CD274    977 non-null    int64 
 11  PIK3CA   977 non-null    int64 
 12  TOP2A    977 non-null    int64 
 13  DCLK1    977 non-null    int64 
 14  MYC      977 non-null    int64 
 15  LRPPRC   977 non-null    int64 
 16  BRCA2    977 non-null    int64 
 17  TP53     977 non-null    int64 
 18  MKI67    977 non-null    int64 
 19  TTN      977 non-null    int64 
 20  CTLA4    977 non-null    int64 
 21  PTEN     977 non-null    int64 
 22  YE

### Imports

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

%run "..\Model\DataHelpers.ipynb"

### Dataset split: training and test data

In [5]:
X, y, X_train, X_test, y_train, y_test, test_case_id = split_data(df, "tnbc", "case_id")

X_train.shape=(781, 30)
X_test.shape=(196, 30)
y_train.shape=(781,)
y_test.shape=(196,)


### Support Vector Machine (SVM)

In [6]:
# Create model

model = SVC(random_state=RANDOM_STATE, probability=True)

# Train the model
model.fit(X_train, y_train)

# Model predictions
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]  # For ROC curves etc.

# Save it in a dataframe, to CSV
predictions = pd.DataFrame({
    "case_id": test_case_id,
    "y_test": y_test,
    "y_pred": y_pred,
    "y_prob": y_prob
})
predictions.to_csv(f"../Data/model_output_{variant}.csv", index=False)

# Evaluate model
print_evaluated_model_accuracy(y_test, y_pred)

Accuracy: 0.94


## Model cross validation

In [7]:
metrics = get_cross_validation_metrics(model, X, y, 5)
test_metrics = get_metrics(y_test, y_pred, y_prob)
test_metrics["fold"] = 0 # Initial test metrics (before cross validation)
test = pd.DataFrame([test_metrics])
test.set_index("fold", inplace=True)

print_validated_model_accuracy(model, metrics)

# Prepend test_metrics to metrics dataframe, export and display
metrics = pd.concat([test, metrics])
metrics.to_csv(f"../Data/model_metrics_{variant}.csv", index=False)
metrics

Model validation for SVC:
[0.9285714285714286, 0.9285714285714286, 0.9384615384615385, 0.9230769230769231, 0.9128205128205128]

Mean accuracy: 0.9263



,accuracy,recall,precision,f1_score,roc_auc,true_positive,true_negative,false_positive,false_negative
fold,,,,,,,,,
0,0.943878,0.652174,0.833333,0.731707,0.975873,15,170,3,8
1,0.928571,0.608696,0.736842,0.666667,0.971852,14,168,5,9
2,0.928571,0.608696,0.736842,0.666667,0.966826,14,168,5,9
3,0.938462,0.565217,0.866667,0.684211,0.962588,13,170,2,10
4,0.923077,0.565217,0.722222,0.634146,0.966127,13,167,5,10
5,0.912821,0.521739,0.666667,0.585366,0.929474,12,166,6,11


# SHAP Example

### Load initjs for visualization

In [44]:
shap.initjs() # necessary for visualization

### With full X_train set, SHAP gives warning, and suggests sampling
### Using 781 background data samples could cause slower run times

#### Kevin: I suggest we use stratified sampling of the X_train set
####        SHAP doesn't supply stratified sampling, sample using other library

In [45]:
shapSample = shap.sample(X_train, 100, random_state=42) # Kevin: use other library to sample as this isn't stratified

In [46]:
targetGenes = ['FYN', 'BRCA1', 'FOXC1', 'TBC1D1', 'LAG3', 'CDK6', 'GATA3', 'CCND1',
       'PRR4', 'EPCAM', 'CD274', 'PIK3CA', 'TOP2A', 'DCLK1', 'MYC', 'LRPPRC',
       'BRCA2', 'TP53', 'MKI67', 'TTN', 'CTLA4', 'PTEN', 'YES1', 'YOD1',
       'TACSTD2', 'ROR1', 'PDCD1', 'SRC', 'DDX3X', 'MALAT1']

explainer = shap.KernelExplainer(model=model.predict_proba, data=shapSample, feature_names=targetGenes) 

### Get values and visualize

In [47]:
# 100 samples = 10 minutes processing time on my laptop
shap_values = explainer.shap_values(X_test)
# this is multiclass so we only visualize the contributions to first class (hence index 0)
shap.force_plot(explainer.expected_value[0], shap_values[..., 0], X_test)

  0%|          | 0/196 [00:00<?, ?it/s]